## **The following cell has some required code to run to get SQL to work on this notebook so be sure to follow along!**

'%' and '%%'' are indicators for magic functions, load_ext allows us to grab an external library not already supported and load it in for use.

We use the 'npx degit' line to grab our database files from their folder location and insert them into a new folder on colab titled Data. Alternatively instead of 'Data', we  can use -f to download directly to the content folder (default location) of colab.

*npx is a call to run a binary from npm, in our case degit. degit is a depth git call to clone a folder within a repository*

Now that we have our databases are loaded in we use the commands on the last two lines to select the database we wish to use. The last selection is the database that is active.

In [ ]:
%load_ext sql
!npx degit The-CEAS-Library/Foundations-of-SQL/Databases Data
%sql sqlite:////content/Data/weather_stations.db
%sql sqlite:////content/Data/rexon_metals.db

# Exercise 1
Write a SQL query to select the columns dew_point, station_pressure, visibility and create a new column that has the wind_speed in terms of meters per second upto one decimal \
- **Hint**: wind_speed_mps = wind_speed*3.6

Solution to Exercise 1

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT dew_point, station_pressure, visibility, wind_speed*3.6 AS wind_speed_mps
FROM station_data

# Exercise 2
Write a SQL query to find the present date by adding your AGE in years, months, and days to your birthyear.

- **Hint**: SELECT DATE('YYYY-MM-DD','+AGE year' ,'+no.of month','+no.of day')

Solution to Exercise 2:

In [ ]:
%%sql
SELECT DATE('1994-11-24', '+29 year', '+4 month', '+10 day')
-- this output will get 2024-4-3

# Exercise 3
#### Data Workflows and Automation

$\;\;\;\;\;\;\;\;$ a. Write a SQL query to filter all records data between year 2009 and 2012 \
$\;\;\;\;\;\;\;\;$ b. Write a SQL query to filter data from the month of May and October of every year \
$\;\;\;\;\;\;\;\;$ c. write a SQL query to filter data where every report_code starts with letter 'C' and has third letter as '9'

Possible Solutions to Exercise 3a:

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE year BETWEEN 2009 and 2012

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE year >= 2009 and year <= 2012

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE year in (2009, 2010, 2011, 2012)

Solution to Exercise 3b:

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE month IN (5, 10)

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE month = 5 or month = 10

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE month % 5 = 0

Solution to Exercise 3c:

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE report_code LIKE 'C_9%'

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT * FROM station_data
WHERE report_code REGEXP '^C.9.*$'

#Exercise 4

Write an SQL Query to Select data yearwise and monthwise with data recorded post 2004 and ordered by latest first and giving data of the sum of windspeed, sum of visibility and maximum visibility

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT year,
month,
SUM(wind_speed) AS total_wind_speed,
SUM(visibility) AS total_visibility,
MAX(visibility) AS Max_visibility
FROM station_data
WHERE year > 2004
GROUP BY year, month
ORDER BY year DESC, month DESC

# Bonus Exercises



## Bonus Exercise 1

Write a SQL query to **JOIN** the 3 tables (CUSTOMER_ORDER, PRODUCT, CUSTOMER) and return columns of customer name, customer state, customer city, customer address, order date, ship date, product name, and the price. \
- Hint: You can perform 2 join operations in the same query. Name, state, city, and address come from the CUSTOMER table, order and ship date come from CUSTOMER_ORDER table, and product description and price come from the PRODUCT table.

In [ ]:
%%sql sqlite:////content/Data/rexon_metals.db
SELECT CUSTOMER.NAME,
CUSTOMER.STATE,
CUSTOMER.CITY,
CUSTOMER.STREET_ADDRESS,
ORDER_DATE,
SHIP_DATE,
PRODUCT.DESCRIPTION as PRODUCT_NAME,
PRODUCT.PRICE

FROM CUSTOMER_ORDER
JOIN CUSTOMER ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
JOIN PRODUCT ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID


## Bonus Exercise 2
Write a SQL query to join all the tables in the weather_stations database (station_data, report_information, station_location), from this merged table we want: the forecaster, state, city, wind speed, and temperature where no tornados occured and sorted chronologically.
    
- **Hint**: city is station_location.place

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT report_information.weather_forecaster,
station_location.state,
station_location.place as city,
station_data.wind_speed,
station_data.temperature
FROM station_Data
INNER JOIN report_information
ON station_data.report_code = report_information.report_code
INNER JOIN station_location
ON station_data.report_code = station_location.report_code
WHERE station_data.tornado = FALSE
ORDER BY year, month, day

## Bonus Exercise 3
This exercise will select many aggregates and return them. We want to select the max precipitation and temperature, and the average report percentage of tornados, thunder, and hail. Only select rows where: 0 < percentage < 100 (exclusive). Group the data by station number and return the first 30 entries.
    
- **Hint**: **WHERE** does not support aggregate operations, try **HAVING** instead. **HAVING** comes after **GROUP BY** in use order.

In [ ]:
%%sql sqlite:////content/Data/weather_stations.db
SELECT max(wind_speed) AS 'Max wind speed',
max(precipitation) AS 'MAX Precipitation',
max(temperature) AS 'MAX Temperature',
avg(tornado)*100,
cast((avg(tornado) * 100) AS INTEGER) || '%' AS 'Percent tornado',
cast((avg(thunder) * 100) AS INTEGER) || '%' AS 'Percent thunder',
cast((avg(hail) * 100) AS INTEGER) || '%' AS 'Percent hail'
FROM STATION_DATA
GROUP BY station_number
HAVING (avg(tornado) * 100 BETWEEN 1 AND 99)
  AND (avg(thunder) * 100 BETWEEN 1 AND 99)
  AND (avg(hail) * 100 BETWEEN 1 AND 99)
LIMIT 30